In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

Connecting Google Drive to Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/My Drive/--path to darknet folder--"
!unzip "/content/drive/My Drive/--path to zipped test folder--"

In [ ]:
%cd /content/darknet-master
!make
!chmod +x ./darknet

In [ ]:
!sudo apt install dos2unix

In [ ]:
!dos2unix ./data/train.txt
!dos2unix ./data/val.txt
!dos2unix ./data/yolo.data
!dos2unix ./data/yolo.names
!dos2unix ./cfg/yolov3_custom_test.cfg

# Predictions

In [ ]:
def get_ltrb(xc,yc,rw,rh):
    l = int((xc-rw/2)*128)
    t = int((yc-rh/2)*128+rh*128)
    r = int(l + rw*128)
    b = int((yc-rh/2)*128)
    return [l,t,r,b]


In [ ]:
cfg_path = "cfg/yolov3_custom_test.cfg"
weights_path = "backup/yolov3_custom_train_2000.weights"
net = cv2.dnn.readNetFromDarknet(cfg_path, weights_path)

In [ ]:
def predict(image, pat_id, f):
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

    net.setInput(blob)
    layerOutputs = net.forward(ln)
    
    threshold = 0.1
    br_det = 0
    ltrb = []
    for output in layerOutputs:
        for detection in output:
            if detection[4] > threshold:
              br_det = br_det + 1
              x, y, w, h = detection[:4]
              ltrb = ltrb + [get_ltrb(x,y,w,h)]

    if br_det == 0:
      line = "{} {}\n".format(pat_id, 'none')
      f.write(line)
    elif br_det > 0:
      ltrb = ltrb[0:2]
      for l in ltrb:
        line = "{} {} {} {} {}\n".format(pat_id, l[0], l[1], l[2], l[3])
        f.write(line)
    return

In [ ]:
%cd /content

In [ ]:
%%time

imgs = os.listdir("dip_test1/p")
output_file_path = "output.txt"
f = open(output_file_path, 'w')
for img in imgs:
  predict(cv2.imread(os.path.join("dip_test1/p", img)), img.split('.')[0], f)
f.close()    